In [1]:
import requests
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

In [3]:

import requests
api_url = "https://debt-api-4301881a2ff8.herokuapp.com/loan/get_all"
res = requests.get(api_url)
data = pd.DataFrame(res.json()["data"])


ConnectionError: HTTPSConnectionPool(host='debt-api-4301881a2ff8.herokuapp.com', port=443): Max retries exceeded with url: /loan/get_all (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x17591b910>: Failed to resolve 'debt-api-4301881a2ff8.herokuapp.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
data.head()

,loan_id,gender,married,dependents,education,self_employed,applicantIncome,coapplicant_income,loan_amount,loan_amount_term,credit_history,property_area,loan_status
0,LP001002,Male,No,0,Graduate,No,5849,0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0,141.0,360.0,1.0,Urban,Y


In [ ]:
# (data[((data['ApplicantIncome'] + data['CoapplicantIncome']) * data['Loan_Amount_Term'] < data['LoanAmount']) & (data['Loan_Status'] == 'Y')])


In [38]:
classification_models = {
    'DecisionTreeClassifier': 'DecisionTreeClassifier',
    'KNNClassifier': 'KNNClassifier',
    'LogisticRegression': 'LogisticRegression',
    'NaiveBayes': 'NaiveBayes',
    'RandomForestClassifier': 'RandomForestClassifier',
    'RandomForestRegressor': 'RandomForestRegressor',
    'SupportVectorClassifier': 'SupportVectorClassifier',
}

In [39]:
model_files = {
            classification_models['DecisionTreeClassifier']: './DecisionTreeClassifier.pkl',
            classification_models['KNNClassifier']: './KNeighborsClassifier.pkl',
            classification_models['LogisticRegression']: './LogisticRegression.pkl',
            classification_models['NaiveBayes']: './GaussianNaiveBayes.pkl',
            classification_models['RandomForestClassifier']: './random_forest_model.pkl',
            # classification_models['SupportVectorClassifier']: './../../files/pkl/SupportVectorClassifier.pkl',
            # classification_models['RandomForestRegressor']: './../../files/pkl/RandomForestRegressor.pkl',
        }

In [40]:

decisionTreeClassifier = pickle.load(open(model_files['DecisionTreeClassifier'] , 'rb'))
kNNClassifier = pickle.load(open(model_files['KNNClassifier'] , 'rb'))
logisticRegression = pickle.load(open(model_files['LogisticRegression'] , 'rb'))
naiveBayes = pickle.load(open(model_files['NaiveBayes'] , 'rb'))
randomForestClassifier = pickle.load(open(model_files['RandomForestClassifier'] , 'rb'))
# supportVectorClassifier = pickle.load(open(model_files['SupportVectorClassifier'] , 'rb'))
# randomForestRegressor = pickle.load(open(model_files['RandomForestRegressor'] , 'rb'))



In [ ]:
# result_SupportVectorClassifier =  pickle.load(open(model_files['SupportVectorClassifier'] , 'rb'))

In [47]:
score = {
    'DecisionTreeClassifier': {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    },
    'KNNClassifier': {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    },
    'LogisticRegression': {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    },
    'NaiveBayes': {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    },
    'RandomForestClassifier': {
        'TP': 0,
        'TN': 0,
        'FP': 0,
        'FN': 0
    },
}

In [46]:
def check(loan_status, prediction, obj, model_name):
    # print(loan_status, prediction)
    if(loan_status == 'Y' and prediction == 1):
        obj[model_name]['TP'] += 1
        # print("TP")
    elif(loan_status == 'N' and prediction == 0):
        obj[model_name]['FP'] += 1
        # print("TN")
    elif(loan_status == 'N' and prediction == 1):
        obj[model_name]['TN'] += 1
        # print("FP")
    elif(loan_status == 'Y' and prediction == 0):
        obj[model_name]['FN'] += 1
        # print("FN")

In [48]:
# sayed bhai
for index, row in data.iterrows():
    if pd.isnull(row['gender']):
        row['gender'] = 'Male'
    if pd.isnull(row['married']):
        row['married'] = 'No'
    if pd.isnull(row['dependents']):
        row['dependents'] = 0
    if pd.isnull(row['education']):
        row['education'] = 'Not Graduate'
    if pd.isnull(row['self_employed']):
        row['self_employed'] = 'No'
    if pd.isnull(row['applicantIncome']):
        row['applicantIncome'] = 0
    if pd.isnull(row['coapplicant_income']):
        row['coapplicant_income'] = 0
    if pd.isnull(row['loan_amount']):
        row['loan_amount'] = 0
    if pd.isnull(row['loan_amount_term']):
        row['loan_amount_term'] = 0
    if pd.isnull(row['credit_history']):
        row['credit_history'] = 0
    if pd.isnull(row['property_area']):
        row['property_area'] = 'Rural'

    input_data_df = pd.DataFrame([row])
    input_data_df = pd.get_dummies(input_data_df, columns=['gender', 'married', 'dependents', 'education', 'self_employed', 'property_area'])
 
    input_data_df = input_data_df.reindex(columns=kNNClassifier.feature_names_in_, fill_value=0)

    dt = decisionTreeClassifier.predict(input_data_df)[0]
    knn = kNNClassifier.predict(input_data_df)[0]
    lr = logisticRegression.predict(input_data_df)[0]
    nb = naiveBayes.predict(input_data_df)[0]
    rf = randomForestClassifier.predict(input_data_df)[0]

    result_decisionTree = int( dt == row['loan_status'])
    result_kNNClassifier = int(knn == row['loan_status'])
    result_logisticRegression = int( lr == row['loan_status'])
    result_naiveBayes = int(nb == row['loan_status'])
    result_randomForest = int(nb == row['loan_status'])

    # 'DecisionTreeClassifier': 'DecisionTreeClassifier',
    # 'KNNClassifier': 'KNNClassifier',
    # 'LogisticRegression': 'LogisticRegression',
    # 'NaiveBayes': 'NaiveBayes',
    # 'RandomForestClassifier': 'RandomForestClassifier',

    check(row['loan_status'], result_decisionTree,score, 'DecisionTreeClassifier')
    check(row['loan_status'], result_kNNClassifier,score, 'KNNClassifier')
    check(row['loan_status'], result_logisticRegression,score, 'LogisticRegression')
    check(row['loan_status'], result_naiveBayes,score, 'NaiveBayes')
    check(row['loan_status'], result_randomForest,score, 'RandomForestClassifier')

    # print(decisionTreeClassifier.predict(input_data_df)[0], kNNClassifier.predict(input_data_df)[0], logisticRegression.predict(input_data_df)[0], naiveBayes.predict(input_data_df)[0], row['loan_status'])
    # print(row['loan_id'],result_KNNClassifier, result_LogisticRegression, result_NaiveBayes, result_RandomForestClassifier, result_SupportVectorClassifier, row['loan_status'])
    # print(result_decisionTree, result_kNNClassifier, result_logisticRegression, result_naiveBayes, row['credit_history'], row['loan_status'])
    # print(row['loan_id'],dt, knn, lr, nb, row['credit_history'], row['loan_status'])


In [49]:
score

{'DecisionTreeClassifier': {'TP': 374, 'TN': 102, 'FP': 90, 'FN': 48},
 'KNNClassifier': {'TP': 411, 'TN': 145, 'FP': 47, 'FN': 11},
 'LogisticRegression': {'TP': 378, 'TN': 96, 'FP': 96, 'FN': 44},
 'NaiveBayes': {'TP': 357, 'TN': 104, 'FP': 88, 'FN': 65},
 'RandomForestClassifier': {'TP': 357, 'TN': 104, 'FP': 88, 'FN': 65}}

In [55]:
def calculate_scores(TP, TN, FP, FN):
    precision = TP / (TP + FP)
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return {"precision": precision, "accuracy": accuracy, "f1_score": f1_score, "recall": recall}


In [58]:
print('DecisionTreeClassifier' ,calculate_scores(score['DecisionTreeClassifier']['TP'], score['DecisionTreeClassifier']['TN'], score['DecisionTreeClassifier']['FP'], score['DecisionTreeClassifier']['FN']))
print('KNNClassifier' ,calculate_scores(score['KNNClassifier']['TP'], score['KNNClassifier']['TN'], score['KNNClassifier']['FP'], score['KNNClassifier']['FN']))
print('LogisticRegression' ,calculate_scores(score['LogisticRegression']['TP'], score['LogisticRegression']['TN'], score['LogisticRegression']['FP'], score['LogisticRegression']['FN']))
print('NaiveBayes' ,calculate_scores(score['NaiveBayes']['TP'], score['NaiveBayes']['TN'], score['NaiveBayes']['FP'], score['NaiveBayes']['FN']))
print('RandomForestClassifier' ,calculate_scores(score['RandomForestClassifier']['TP'], score['RandomForestClassifier']['TN'], score['RandomForestClassifier']['FP'], score['RandomForestClassifier']['FN']))


DecisionTreeClassifier {'precision': 0.8060344827586207, 'accuracy': 0.7752442996742671, 'f1_score': 0.8442437923250564, 'recall': 0.8862559241706162}
KNNClassifier {'precision': 0.8973799126637555, 'accuracy': 0.9055374592833876, 'f1_score': 0.9340909090909091, 'recall': 0.9739336492890995}
LogisticRegression {'precision': 0.7974683544303798, 'accuracy': 0.7719869706840391, 'f1_score': 0.84375, 'recall': 0.8957345971563981}
NaiveBayes {'precision': 0.802247191011236, 'accuracy': 0.750814332247557, 'f1_score': 0.823529411764706, 'recall': 0.8459715639810427}
RandomForestClassifier {'precision': 0.802247191011236, 'accuracy': 0.750814332247557, 'f1_score': 0.823529411764706, 'recall': 0.8459715639810427}


In [ ]:
def findChange(row, prediction):
    if row['loan_status'] == 'Y':
        return 1
    else:
        return 0